In [16]:
from dataclasses import dataclass
from typing import List, Tuple, Optional
from itertools import cycle
import random
import json

In [2]:
@dataclass
class Minion:
    """Minion characteristics and stats"""
    name: str
    attack: int
    health: int
    alignment: str
    types: List[str]
    level: int
    upgraded: bool = False
    support: Optional[dict] = None
    ranged: bool = False
    flying: bool = False
    slay: bool = False

    def take_damage(self, damage: int) -> None:
        self.health -= damage

    def on_death(self):
        pass

In [3]:
def can_attack(minion: Optional[Minion]) -> bool:
    return minion is not None and minion.attack > 0

In [4]:
supported_by = {
    0: [4],
    1: [4, 5],
    2: [5, 6],
    3: [6],
    4: [0, 1],
    5: [1, 2],
    6: [2, 3]
}

In [42]:
class Player:
    """Board position details"""
    name: str
    positions: List[Optional[Minion]]
    current_position: int = 0
    last_uid: Optional[int] = None 

    def __init__(self, name: str, minion_names: Tuple[Optional[str]]):
        self.name = name
        self.positions = [Minion(**ALL_MINIONS[name]) if name else None for name in minion_names]

    def next_position(self) -> Optional[int]:
        """Finds the position of the next attacking minion, or None if no minions can attack"""

        if not any(can_attack(p) for p in self.positions):
            return None

        idx = self.current_position
        minion = self.positions[idx]
        if can_attack(minion) and (self.last_uid is None or id(minion) != self.last_uid):
            return self.current_position

        while True:
            idx = (idx + 1) % len(self.positions)
            if can_attack(self.positions[idx]):
                return idx

    def resolve_supports(self) -> None:
        for position, minion in enumerate(self.positions):
            if position < 4 or minion is None or minion.support is None:
                continue
            for p in supported_by[position]:
                supported = self.positions[p]
                if supported is not None and (minion.support['type'] == 'All' or minion.support['type'] in supported.types):
                    for stat in minion.support['stats']:
                        setattr(supported, stat, getattr(supported, stat) + minion.support['value'])

    def get_defender(self) -> Minion:
        try: 
            return random.choice([minion for minion in self.positions[:4] if minion is not None])
        except IndexError:
            return random.choice([minion for minion in self.positions[4:] if minion is not None])

    def attack(self, opponent) -> None:
        self.current_position = self.next_position()
        if self.current_position is None:
            return
        attacker = self.positions[self.current_position]
        defender = opponent.get_defender()
        print(f"{self.name}'s {attacker.name} attacks {defender.name}!")
        defender.take_damage(attacker.attack)
        if not attacker.ranged:
            attacker.take_damage(defender.attack)
        self.last_uid = id(attacker)

    def check_deaths(self) -> None:
        for i, minion in enumerate(self.positions):
            if minion is None:
                continue
            if minion.health <= 0:
                minion.on_death()
                self.positions[i] = None

    @property
    def has_empty_board(self) -> bool:
        return not any(self.positions)
            


In [118]:
def battle(p1, p2):
    p1.resolve_supports()
    p2.resolve_supports()
    for player, opponent in cycle([(p1, p2), (p2, p1)]):
        player.attack(opponent)
        player.check_deaths()
        opponent.check_deaths()
        if player.has_empty_board and opponent.has_empty_board:
            print("Draw!")
            break
        elif player.has_empty_board:
            print(f"{opponent.name} wins!")
            break
        elif opponent.has_empty_board:
            print(f"{player.name} wins!")
            break
        elif sum(minion.attack for minion in filter(None, player.positions + opponent.positions)) == 0:
            print("Draw!")
            break

In [119]:
p1 = Player("Player 1", ('Black Cat', None, None, None, 'Mad Mim', None, None))
p2 = Player("Player 2", ('Rainbow Unicorn', None, None, None, None, None, None))

battle(p1, p2)

Player 1's Black Cat attacks Rainbow Unicorn!
Player 2's Rainbow Unicorn attacks Mad Mim!
Player 2's Rainbow Unicorn attacks Mad Mim!
Player 2's Rainbow Unicorn attacks Mad Mim!
Player 2 wins!


In [120]:
p1 = Player("Player 1", ('Black Cat', None, None, None, None, None, None))
p2 = Player("Player 2", ('Black Cat', None, None, None, None, None, None))

battle(p1, p2)

Player 1's Black Cat attacks Black Cat!
Draw!


In [121]:
p1 = Player("Player 1", ('Black Cat', None, None, None, 'Mad Mim', None, None))
p2 = Player("Player 2", ('Black Cat', None, None, None, 'Mad Mim', None, None))

battle(p1, p2)

Player 1's Black Cat attacks Black Cat!
Draw!


In [133]:
p1 = Player("Player 1", ('Sherwood Sureshot', 'Black Cat', 'Golden Chicken', None, 'Mad Mim', None, None))
p2 = Player("Player 2", ('B-a-a-d Billy Gruff', 'Tiny', 'Blind Mouse', None, 'Baby Root', None, None))

battle(p1, p2)

Player 1's Sherwood Sureshot attacks Blind Mouse!
Player 2's B-a-a-d Billy Gruff attacks Black Cat!
Player 1's Golden Chicken attacks Tiny!
Player 2's Tiny attacks Sherwood Sureshot!
Player 2's B-a-a-d Billy Gruff attacks Mad Mim!
Player 2's B-a-a-d Billy Gruff attacks Mad Mim!
Player 2 wins!


In [6]:
import json

with open('minions.json') as f:
    ALL_MINIONS = json.loads(f.read())

cat1 = Minion(**ALL_MINIONS['Black Cat'])
cat2 = Minion(**ALL_MINIONS['Black Cat'])
assert cat1 is not cat2

In [7]:
p1 = Player(('Black Cat', None, None, None, 'Mad Mim', None, None))
p1.resolve_supports()
p2 = Player(('Rainbow Unicorn', None, None, None, None, None, None))

p1.attack(p2)
p1.check_deaths()
p2.check_deaths()

p2.attack(p1)
p1.attack(p2)

p2.attack(p1)
p1.attack(p2)

p2.attack(p1)
p1.attack(p2)

In [8]:
print(p1.positions)
p2.positions

[None, None, None, None, Minion(name='Mad Mim', attack=0, health=0, alignment='Evil', types=['Mage'], level=2, upgraded=False, support={'stats': ['attack'], 'value': 3, 'type': 'All'}, ranged=False, flying=False, slay=False), None, None]


[Minion(name='Rainbow Unicorn', attack=1, health=1, alignment='Good', types=['Animal'], level=2, upgraded=False, support=None, ranged=False, flying=False, slay=False),
 None,
 None,
 None,
 None,
 None,
 None]

In [12]:
p2.current_position

0

In [15]:
player = Player(('Baby Dragon', None, 'Black Cat', None, 'Mad Mim', 'Baby Root', None))
player.resolve_supports()
assert player.positions[0].attack == 6
assert player.positions[0].health == 2
assert player.positions[2].attack == 1
assert player.positions[2].health == 4

In [13]:
# test next_position

player = Player(('Mad Mim', None, None, None, 'Baby Dragon', None, None))
assert player.next_position() == 4
player.current_position = 4
player.last_uid = id(player.positions[4])
assert player.next_position() == 4

player = Player(('Mad Mim', None, None, None, None, None, None))
assert player.next_position() is None

player = Player(('Baby Dragon', None, None, None, 'Black Cat', None, None))
assert player.next_position() == 0
player.current_position = 0
player.last_uid = id(player.positions[0])
assert player.next_position() == 4
player.current_position = 4
player.last_uid = id(player.positions[4])
assert player.next_position() == 0